In [3]:
import pandas as pd

# Load the data
df = pd.read_csv('../Data_Cleanup/outCSV/Clean_Weather_Set.csv',)

# Show the first few rows of the DataFrame
df.head()

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
0,Burbank,2015-01-01 08:53:00,3.0,33.0,Fair,989.11,0.0,0.0,3.0
1,Burbank,2015-01-01 09:53:00,2.0,33.0,Fair,989.77,0.0,0.0,2.0
2,Burbank,2015-01-01 10:53:00,2.0,33.0,Fair,989.44,0.0,0.0,2.0
3,Burbank,2015-01-01 11:53:00,1.0,33.0,Fair,989.77,0.0,0.0,1.0
4,Burbank,2015-01-01 12:53:00,1.0,33.0,Fair,990.10,0.0,0.0,1.0


In [5]:
# Convert 'timestamp' to datetime and round to the nearest hour
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.round('H')

# Define numeric and non-numeric columns
numeric_cols = ['temperature', 'cloud_cover', 'pressure', 'windspeed', 'precipitation', 'felt_temperature']
non_numeric_cols = ['city', 'cloud_cover_description']

# Compute mean for numeric columns and mode for non-numeric columns
df_grouped = df.groupby([pd.Grouper(key='timestamp'), 'city']).agg(
    {nc: 'mean' for nc in numeric_cols},
    {nnc: lambda x: pd.Series.mode(x)[0] for nnc in non_numeric_cols}
)

# Reset index
df_grouped = df_grouped.reset_index()

df_grouped.head()

,timestamp,city,temperature,cloud_cover,pressure,windspeed,precipitation,felt_temperature
0,2015-01-01 09:00:00,Burbank,3.0,33.0,989.11,0.0,0.0,3.0
1,2015-01-01 10:00:00,Burbank,2.0,33.0,989.77,0.0,0.0,2.0
2,2015-01-01 11:00:00,Burbank,2.0,33.0,989.44,0.0,0.0,2.0
3,2015-01-01 12:00:00,Burbank,1.0,33.0,989.77,0.0,0.0,1.0
4,2015-01-01 13:00:00,Burbank,1.0,33.0,990.10,0.0,0.0,1.0


In [6]:
# Compute mean for numeric columns
numeric_agg = df.groupby(['timestamp', 'city'])[numeric_cols].mean().reset_index()

# Compute mode for non-numeric columns
non_numeric_agg = df.groupby(['timestamp', 'city'])[non_numeric_cols].agg(pd.Series.mode).reset_index()

# Merge the results
df_merged = pd.merge(numeric_agg, non_numeric_agg, on=['timestamp', 'city'])

df_merged.head()


,timestamp,city,temperature,cloud_cover,pressure,windspeed,precipitation,felt_temperature,cloud_cover_description
0,2015-01-01 09:00:00,Burbank,3.0,33.0,989.11,0.0,0.0,3.0,Fair
1,2015-01-01 10:00:00,Burbank,2.0,33.0,989.77,0.0,0.0,2.0,Fair
2,2015-01-01 11:00:00,Burbank,2.0,33.0,989.44,0.0,0.0,2.0,Fair
3,2015-01-01 12:00:00,Burbank,1.0,33.0,989.77,0.0,0.0,1.0,Fair
4,2015-01-01 13:00:00,Burbank,1.0,33.0,990.10,0.0,0.0,1.0,Fair


In [7]:
df_merged.to_csv("../Data_Cleanup/outCSV/YusufMerge.csv")